In [4]:
def compute_dyncut(data,nbClusters,children_map):
    nbVertices = max(children_map)
    inf = float("inf")
    dp = np.zeros((nbClusters+1,nbVertices+1)) + inf
    lcut = np.zeros((nbClusters+1,nbVertices+1))

    for i in range(0,dp.shape[1]):
        dp[1,i] = compute_intra_variance(i,children_map,data)

    root = max(children_map)
    for vertex in range(len(data),root+1):
        left_child, right_child = children_map[vertex]
        for k in range(2,nbClusters+1):
            vmin = inf
            kl_min = -1
            for kl in range(1,k):
                v = dp[kl,left_child] + dp[k-kl,right_child]
                if v < vmin:
                    vmin = v
                    kl_min = kl

            dp[k,vertex] = vmin
            lcut[k,vertex] = kl_min
            
    return dp,lcut

In [5]:
def build_dict_tree(linkage_matrix):
    tree = {}
    n = linkage_matrix.shape[0]+1
    for i in range(0,n-1):
        tree[linkage_matrix[i,0]] = n+i
        tree[linkage_matrix[i,1]] = n+i
    return tree

def build_children_map(tree):
    children_map = {}
    for k, v in tree.items():
        children_map[v] = children_map.get(v, [])
        children_map[v].append(int(k))
    return children_map

def build_children(vertex,children_map):
    children = []
    if vertex in children_map:
        left_child, right_child = children_map[vertex]
        if left_child in children_map:
            children.extend(build_children(left_child,children_map))
        else:
            children.extend([left_child])

        if right_child in children_map:
            children.extend(build_children(right_child,children_map))
        else:
            children.extend([right_child])
    
    return children

def get_var(data,subpop):
    intravar = 0
    center = np.mean(data[subpop],axis=0)
    for elem in subpop:
        x = data[elem] - center
        intravar += np.dot(x,x)
    return intravar

def compute_intra_variance(vertex,children_map,data):
    children = build_children(vertex,children_map)
    intravar = 0
    if children:
        intravar = get_var(data,children)
        
    return intravar

def compute_centers(data,target):
    centers = []
    for i in set(target):
        id_pts = [index for index,value in enumerate(target) if value == i]
        centers.append(np.mean(data[id_pts],axis=0))
        
    return centers

def compute_flat_dyn_clusters(cur_vertex,k,lcut,children_map):
    clusters = []
    #leaf
    if k == 1 and not cur_vertex in children_map:
        clusters.append([cur_vertex])
    #one cluster left, get the leaves
    if k == 1 and cur_vertex in children_map:
        leaves = build_children(cur_vertex,children_map)
        clusters.append(leaves)
    #recurse in left and right subtrees
    if k > 1:
        if cur_vertex in children_map:
            left_child,right_child = children_map[cur_vertex]
            clusters.extend(compute_flat_dyn_clusters(left_child,int(lcut[k,cur_vertex]),lcut,children_map))
            clusters.extend(compute_flat_dyn_clusters(right_child,int(k-lcut[k,cur_vertex]),lcut,children_map))

    return clusters

def compute_flat_cut_clusters(nbClusters,linkage_matrix):
    flat = fcluster(linkage_matrix,nbClusters,'maxclust')
    flat_clusters = []
    for i in range(1,len(set(flat))+1):
        flat_clusters.append( [index for index,value in enumerate(flat) if value == i] )
    
    return flat_clusters

In [6]:
from sklearn import datasets
iris = datasets.load_iris()
X = iris.data
Y = iris.target

In [7]:
def bench_methods(data,nbClusters,methods):
    d = pdist(data)
    for method in methods:
        if method in ['centroid','ward','median']:
            linkage_matrix = linkage(data,method)
        else:
            linkage_matrix = linkage(d,method)
        tree = build_dict_tree(linkage_matrix)
        children_map = build_children_map(tree)
        dp,lcut = compute_dyncut(data,nbClusters,children_map)
        flat_dyn_clusters = compute_flat_dyn_clusters(max(children_map),nbClusters,lcut,children_map)
        flat_cut_clusters = compute_flat_cut_clusters(nbClusters,linkage_matrix)
        
        tot_dyn = 0
        tot_cut = 0
        for i in range(0,nbClusters):
            tot_dyn += get_var(data,flat_dyn_clusters[i])
            tot_cut += get_var(data,flat_cut_clusters[i])
        
        print("method:",method)
        print("intra-variance:", "(DP)",tot_dyn,"\t(cst height)",tot_cut)
        print("\n")

In [31]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

nbClusters = 20
methods = ['single','complete','average','weighted','centroid','median','ward']

data = iris.data
nbClusters = 20
methods = ['single']

In [32]:
len(data)

150

In [35]:
print(len(d))
print(len(data)**2/2)

11175
11250.0


In [14]:
d = pdist(data)
for method in methods:
    if method in ['centroid','ward','median']:
        linkage_matrix = linkage(data,method)
    else:
        linkage_matrix = linkage(d,method)
    tree = build_dict_tree(linkage_matrix)
    children_map = build_children_map(tree)
    dp,lcut = compute_dyncut(data,nbClusters,children_map)
    flat_dyn_clusters = compute_flat_dyn_clusters(max(children_map),nbClusters,lcut,children_map)
    flat_cut_clusters = compute_flat_cut_clusters(nbClusters,linkage_matrix)

    tot_dyn = 0
    tot_cut = 0
    for i in range(0,nbClusters):
        tot_dyn += get_var(data,flat_dyn_clusters[i])
        tot_cut += get_var(data,flat_cut_clusters[i])

    print("method:",method)
    print("intra-variance:", "(DP)",tot_dyn,"\t(cst height)",tot_cut)
    print("\n")

method: single
intra-variance: (DP) 38.34805128205129 	(cst height) 46.142062246963576




In [37]:
linkage_matrix.shape

(149, 4)

In [39]:
len(tree)

298

In [16]:
tree

{101.0: 150,
 142.0: 150,
 7.0: 151,
 39.0: 151,
 0.0: 152,
 17.0: 152,
 9.0: 153,
 34.0: 153,
 128.0: 154,
 132.0: 154,
 10.0: 155,
 48.0: 155,
 40.0: 156,
 152.0: 156,
 4.0: 157,
 37.0: 157,
 19.0: 158,
 21.0: 158,
 156.0: 159,
 157.0: 159,
 29.0: 160,
 30.0: 160,
 57.0: 161,
 93.0: 161,
 80.0: 162,
 81.0: 162,
 116.0: 163,
 137.0: 163,
 8.0: 164,
 38.0: 164,
 46.0: 165,
 158.0: 165,
 151.0: 166,
 159.0: 166,
 49.0: 167,
 166.0: 167,
 27.0: 168,
 28.0: 168,
 1.0: 169,
 153.0: 169,
 3.0: 170,
 47.0: 170,
 82.0: 171,
 92.0: 171,
 95.0: 172,
 96.0: 172,
 127.0: 173,
 138.0: 173,
 2.0: 174,
 170.0: 174,
 45.0: 175,
 169.0: 175,
 12.0: 176,
 175.0: 176,
 167.0: 177,
 168.0: 177,
 160.0: 178,
 176.0: 178,
 99.0: 179,
 172.0: 179,
 63.0: 180,
 91.0: 180,
 65.0: 181,
 75.0: 181,
 25.0: 182,
 178.0: 182,
 69.0: 183,
 162.0: 183,
 123.0: 184,
 126.0: 184,
 112.0: 185,
 139.0: 185,
 174.0: 186,
 182.0: 186,
 94.0: 187,
 179.0: 187,
 88.0: 188,
 187.0: 188,
 66.0: 189,
 84.0: 189,
 78.0: 190,
 1

In [15]:
children_map

{150: [101, 142],
 151: [7, 39],
 152: [0, 17],
 153: [9, 34],
 154: [128, 132],
 155: [10, 48],
 156: [40, 152],
 157: [4, 37],
 158: [19, 21],
 159: [156, 157],
 160: [29, 30],
 161: [57, 93],
 162: [80, 81],
 163: [116, 137],
 164: [8, 38],
 165: [46, 158],
 166: [151, 159],
 167: [49, 166],
 168: [27, 28],
 169: [1, 153],
 170: [3, 47],
 171: [82, 92],
 172: [95, 96],
 173: [127, 138],
 174: [2, 170],
 175: [45, 169],
 176: [12, 175],
 177: [167, 168],
 178: [160, 176],
 179: [99, 172],
 180: [63, 91],
 181: [65, 75],
 182: [25, 178],
 183: [69, 162],
 184: [123, 126],
 185: [112, 139],
 186: [174, 182],
 187: [94, 179],
 188: [88, 187],
 189: [66, 84],
 190: [78, 180],
 191: [23, 26],
 192: [42, 164],
 193: [53, 89],
 194: [74, 97],
 195: [186, 192],
 196: [11, 195],
 197: [6, 196],
 198: [35, 177],
 199: [155, 198],
 200: [43, 191],
 201: [73, 190],
 202: [70, 173],
 203: [199, 200],
 204: [197, 203],
 205: [110, 147],
 206: [120, 143],
 207: [136, 148],
 208: [58, 181],
 209: [1

In [25]:
import matplotlib.pyplot as plt
%matplotlib qt 

In [29]:
plt.imshow(dp)
plt.show()

In [46]:
plt.imshow(lcut)
plt.colorbar()
plt.show()

In [42]:
lcut

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [19]:
print(lcut)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 1. 1. 1.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]
